# Search Based Software Engineering
## Exercise 01 , Task3 -  Simulated Annealing on Traveling Salesman Problem
### Team: 
### 1. Sagar Nagaraj Simha - sagar.nagaraj.simha[at]uni-weimar.de
####     Matrikel Nummer - 120797, Degree - Master's Computer Science for Digital Media
### 2. Mohd Saif Khan - mohd.saif.khan[at]uni-weimar.de 
####     Matrikel Nummer - 120803, Degree - Master's Human Computer Interaction

The Traveling Salesman Problem (TSP) is given by the following question: *“Given is a list of cities and distances between each pair of cities - what is the shortest route that visits each city and returns to the original city?”*

The TSP is an **NP-Hard-Problem** which does not mean an instance of the  problem will be hard to solve. It means, there does not exist an algorithm that produces the best solution in polynomial time. We can not make predictions about how long it might take to find the best solution.

But, we can find a good solution which might not be the best solution. It is ok to find a route amongst 1000 cities that is only few miles longer than the best route. Particularly, if it would take an inordinate amount amount of computing time to get from our good solution to the best solution.

![germany TSP](./TSP_Deutschland_3.png)

## Representation of the Problem

<img src="./Graph_TSP.png" align="left">
A TSP can be modelled as an undirected weighted graph:
        - cities = vertices
        - paths between cities = edges
        - distance of a path = weight of an edge
<!--![graph](./Graph_TSP.png)-->

This graph can be represented as an **Adjacency matrix**:


|  \     | A     | B     | C     | D     |
| :---:  | :---: | :---: | :---: | :---: |
| **A**  |  0    | 20    | 42    | 35    |
| **B**  | 20    | 0     | 30    | 34    |
| **C**  | 42    | 30    | 0     | 12    |
| **D**  | 35    | 34    | 12    | 0     |

But how do we get the distances between cities if we only got the coordinates for each city?

Each city is represented by a cartesian koordinate P

$ P = (p_x, p_y) $

### Euclidean Distance

Euclidean distance between two points P<sub>1</sub> = (x<sub>1</sub>, y<sub>1</sub>) and P<sub>2</sub> = (x<sub>2</sub>, y<sub>2</sub>) is:

$d(P_{1},P_{2}) = \sqrt{(x_{1} - x_{2})^2 + (y_{1} - y_{2})^2}$

In [2]:
from itertools import permutations
import datetime

In [3]:
def distance(p1, p2):
    """
    Returns the Euclidean distance of two points in the Cartesian Plane.

    >>> distance([3,4],[0,0])
    5.0
    
    """
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) ** 0.5

In [4]:
def total_distance(points):
    """
    Returns the length of the path passing throught
    all the points in the given order.

    >>> total_distance([[1,2],[4,6]])
    5.0
    >>> total_distance([[3,6],[7,6],[12,6]])
    9.0
    """
    return sum([distance(point, points[index + 1]) for index, point in enumerate(points[:-1])])

#### Adjacency Matrix

We also need to specify a "distance matrix" that we can use to keep track of distances between cities. To generate a distance matrix for a set of (x,y) coordinates we will use the following function:

In [5]:
def cartesian_matrix(coordinates):
    '''
    Creates a distance matrix for the city coords using straight line distances
    computed by the Euclidean distance of two points in the Cartesian Plane.
    '''
    matrix = {}
    for i, p1 in enumerate(coordinates):
        for j, p2 in enumerate(coordinates):
            matrix[i,j] = distance(p1,p2)
    return matrix

This function takes a list of (x,y) tuples and outputs a dictionary that contains the distance between any pair of cities:

In [6]:
m = cartesian_matrix([(0,0), (1,0), (1,1)])
for k, v in m.items():
    print(k, v)
print()
print(m[2,0])

(0, 0) 0.0
(0, 1) 1.0
(0, 2) 1.4142135623730951
(1, 0) 1.0
(1, 1) 0.0
(1, 2) 1.0
(2, 0) 1.4142135623730951
(2, 1) 1.0
(2, 2) 0.0

1.4142135623730951


\[2,0\] gives the distance between the city with number 2 and the city with  number 0.
In our case the result of \[2,0\] is the same for \[0,2\], but for other TSPs this may not be the case (for example if a street between two cities is only one way - we have to take another route)

#### Read City Coordinates from File

In [7]:
def read_coords(file_handle):
    coords = []
    for line in file_handle:
        x,y = line.strip().split(',')
        coords.append((float(x), float(y)))
    return coords

with open('city100.txt', 'r') as coord_file:
    coords = read_coords(coord_file)
matrix = cartesian_matrix(coords)

On real world problems it may be more complicated to generate a distance matrix - you might need to take a map and calculate the real distances between cities.

#### Compute the Total Distance

In [8]:
def tour_length(matrix, tour):
    """Sum up the total length of the tour based on the distance matrix"""
    result = 0
    num_cities = len(list(tour))
    for i in range(num_cities):
        j = (i+1) % num_cities
        city_i = tour[i]
        city_j = tour[j]
        result += matrix[city_i, city_j]
    return result

#### Implementing Tweak Operators

We will implement the two tweak operators as generator functions that will return all possible versions of a route that can be made in one step of the generator (in a random order).

Generators are iterators which can be only iterated once.
They generate values on the fly and do not store them in memory.
By using a generator function, we can get each possiblility and perhaps decide to not generate any more variations.
This saves the overhead of generating all combinations at once.

In [9]:
import random

def all_pairs(size, shuffle = random.shuffle):
    r1 = list(range(size))
    r2 = list(range(size))
    if shuffle:
        shuffle(r1)
        shuffle(r2)
    for i in r1:
        for j in r2:
            yield(i,j) # yield is an iterator function
            # for each call of the generator it returns the next value in yield

In [10]:
from copy import deepcopy

# Tweak 1
def swapped_cities(tour):
    """
    Generator to create all possible variations where two 
    cities have been swapped
    """
    ap = all_pairs(len(tour))
    for i,j in ap:
        if i < j:
            copy = deepcopy(tour)
            copy[i], copy[j] = tour[j], tour[i]
            yield copy

# Tweak 2
def reversed_sections(tour):
    """
    Generator to return all possible variations where the
    section between two cities are swapped.
    It preserves entire sections of a route,
    yet still affects the ordering of multiple cities in one go.
    """
    ap = all_pairs(len(tour))
    for i,j in ap:
        if i != j:
            #print("indices from:",i, "to", j)
            copy = deepcopy(tour)
            if i < j:
                copy[i:j+1] = reversed(tour[i:j+1])
            else:
                copy[i+1:] = reversed(tour[:j])
                copy[:j] = reversed(tour[i+1:])
            if copy != tour: # not returning same tour
                yield copy
# usage
print("start tour swap:",[1,2,3,4])
for tour in swapped_cities([1,2,3,4]):
    print(tour)
print()
print("start tour reverse section:",[1,2,3,4])
for tour in reversed_sections([1,2,3,4]):
    print(tour)

start tour swap: [1, 2, 3, 4]
[1, 3, 2, 4]
[1, 4, 3, 2]
[1, 2, 4, 3]
[3, 2, 1, 4]
[4, 2, 3, 1]
[2, 1, 3, 4]

start tour reverse section: [1, 2, 3, 4]
[4, 3, 1, 2]
[1, 3, 2, 4]
[1, 4, 3, 2]
[4, 1, 2, 3]
[1, 2, 4, 3]
[4, 2, 3, 1]
[3, 2, 1, 4]
[4, 3, 2, 1]
[2, 1, 3, 4]
[3, 4, 2, 1]
[2, 3, 4, 1]


In [11]:
# New Tweak Operator - rev_city()

In [12]:
import random
import copy

def rev_city(tour):
    new_tour = copy.deepcopy(tour)
    i = random.randint(0,len(tour))
    j = random.randint(0,len(tour))
    if(i!=j):
        new_tour[min(i,j) : max(i,j)] = tour[min(i,j) : max(i,j)][::-1]
    return new_tour

In [13]:
#Example
tour = [i for i in range(20)]
print(tour)
print(tour[::-1])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


In [14]:
#Example
test = rev_city(tour)
print(test)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [15]:
def init_random_tour(tour_length):
    tour = list(range(tour_length))
    random.shuffle(list(tour))
    return tour

init_function = lambda: init_random_tour(len(coords))
objective_function = lambda tour: tour_length(matrix, tour)

In [16]:
# The temperature function
from math import exp
import random
def temperatureDifference(temperature,next_score,Sbest_score):
    temperature_probability=exp(-abs(Sbest_score-next_score)/temperature) 
    
    if random.random()<temperature_probability:
        return True
    else:
        return False

In [17]:
def simulatedAnnealing(init_function, move_operator, objective_function, max_evaluations,temperature,alpha):
    '''
    Simulated Annealing
    '''
    Sbest = init_function()
    Sbest_score = objective_function(Sbest)
    best=Sbest
    best_score=Sbest_score
    
    num_evaluations = 1
    
    while num_evaluations < max_evaluations:
        
        next = move_operator(Sbest)
        
        next_score = objective_function(next)
        
        num_evaluations += 1
        if (next_score < Sbest_score) or temperatureDifference(temperature,next_score,Sbest_score):
            Sbest = next
            Sbest_score = next_score
        
        temperature=temperature*alpha
        
        if Sbest_score<best_score:
            best=Sbest
            best_score=Sbest_score
            
    return (num_evaluations, best_score, best)

In [18]:
from PIL import Image, ImageDraw, ImageFont

def write_tour_to_img(coords, tour, title, img_file):
    padding = 20
    # shift all coords in a bit
    coords = [(x+padding,y+padding) for (x,y) in coords]
    maxx, maxy = 0,0
    for x,y in coords:
        maxx = max(x,maxx)
        maxy = max(y,maxy)
    maxx += padding
    maxy += padding
    img = Image.new("RGB",(int(maxx), int(maxy)), color=(255,255,255))
    
    font=ImageFont.load_default()
    d=ImageDraw.Draw(img);
    num_cities = len(tour)
    for i in range(num_cities):
        j = (i+1) % num_cities
        city_i = tour[i]
        city_j = tour[j]
        x1,y1 = coords[city_i]
        x2,y2 = coords[city_j]
        d.line((int(x1), int(y1), int(x2), int(y2)), fill=(0,0,0))
        d.text((int(x1)+7, int(y1)-5), str(i), font=font, fill=(32,32,32))
    
    
    for x,y in coords:
        x,y = int(x), int(y)
        d.ellipse((x-5, y-5, x+5, y+5), outline=(0,0,0), fill=(196,196,196))
    
    d.text((1,1), title, font=font, fill=(0,0,0))
    
    del d
    img.save(img_file, "PNG")

In [19]:
def reload_image_for_jupyter(filename):
    # pick a random integer with 1 in 2 billion chance of getting the same
    # integer twice
    import random
    __counter__ = random.randint(0,2e9)

    # now use IPython's rich display to display the html image with the
    # new argument
    from IPython.display import HTML, display
    display(HTML('<img src="./'+filename+'?%d" alt="Schema of adaptive filter" height="100">' % __counter__))


In [20]:
def do_simanneal_evaluations(evaluations,temperature,alpha,move_operator = swapped_cities):
    max_evaluations = evaluations
    then = datetime.datetime.now()
    num_evaluations, best_score, best = simulatedAnnealing(init_function, move_operator, objective_function, max_evaluations,temperature,alpha)
    now = datetime.datetime.now()

    print("computation time ", now - then)
    print("best score:", best_score)
    print("best route:", best)
    filename = "testSimAnn"+str(max_evaluations)+".PNG"
    write_tour_to_img(coords, best, filename, open(filename, "ab"))
    reload_image_for_jupyter(filename)

In [24]:
print("Check")

Check


In [25]:
move_operator = rev_city
max_evaluations = 50000
temperature = 30000
alpha=0.995
do_simanneal_evaluations(max_evaluations,temperature,alpha,move_operator)

computation time  0:00:04.801525
best score: 4129.584444739712
best route: [4, 42, 2, 12, 93, 6, 45, 37, 58, 72, 95, 76, 55, 94, 32, 15, 53, 71, 47, 65, 48, 51, 3, 84, 54, 16, 36, 22, 67, 88, 18, 34, 21, 10, 96, 30, 9, 73, 86, 56, 39, 35, 0, 68, 5, 92, 50, 99, 66, 44, 77, 69, 60, 82, 87, 81, 33, 46, 70, 24, 89, 29, 17, 27, 91, 40, 98, 25, 7, 57, 59, 79, 20, 31, 19, 14, 8, 74, 83, 49, 43, 38, 23, 63, 1, 41, 61, 26, 78, 90, 97, 52, 11, 13, 64, 62, 75, 80, 85, 28]


In [26]:
move_operator = rev_city
max_evaluations = 100000
temperature = 30000
alpha=0.95
do_simanneal_evaluations(max_evaluations,temperature,alpha,move_operator)

computation time  0:00:09.940067
best score: 4283.227253269815
best route: [45, 6, 37, 58, 72, 95, 76, 55, 94, 32, 15, 71, 53, 13, 11, 52, 47, 65, 48, 84, 54, 16, 36, 61, 41, 26, 78, 3, 51, 97, 90, 1, 63, 23, 38, 14, 19, 31, 50, 9, 30, 10, 96, 88, 67, 22, 18, 34, 21, 73, 86, 56, 39, 35, 0, 68, 5, 92, 66, 44, 77, 99, 20, 79, 59, 82, 69, 60, 87, 33, 24, 70, 27, 17, 29, 89, 91, 40, 98, 46, 81, 7, 57, 8, 43, 49, 74, 83, 80, 85, 28, 4, 42, 25, 2, 93, 12, 75, 62, 64]
